# Lecture 6: Bayes' Rule and the Law of Total Probability
***

In this notebook we'll get some more practice with conditional probabilities, total probability, the product rule, and now Bayes' Rule.  We'll also see how we can do some simple random simulations using Numpy to verify our results. 

We'll need Numpy for this notebook, so let's load it. 

In [5]:
import numpy as np 

### Example - Estimating the Probability of Drawing Marbles from Bags in a Box
***
In class we computed solved the following problem: 

Suppose I have two bags of marbles.  The first bag contains 6 white marbles and 4 black marbles.  The second bag contains 3 white marbles and 7 black marbles.  Now suppose I put the two bags in a box.  If I close my eyes, grab a bag from the box, and then grab a marble from the bag, what is the probability that it is black if I'm twice as likely to grab the first bag as the second bag? 

The following code runs a simple simulation to estimate the probability of drawing a marble of a particular color.  Run the code and verify that it agrees with the by-hand computation. 

In [2]:
bag1 = {'marbles' : np.array(["black", "white"]), 'probs' : np.array([4/10, 6/10])}
bag2 = {'marbles' : np.array(["black", "white"]), 'probs' : np.array([7/10, 3/10])}
box  = {'bags' : np.array([bag1, bag2]), 'probs' : np.array([2/3, 1/3])}

def sample_marble(box):
    # randomly choose a bag 
    bag = np.random.choice(box['bags'], p = box['probs'])
    # randomly choose a marble 
    return np.random.choice(bag['marbles'], p = bag['probs'])

def probability_of_color(color, box, num_samples=1000):
    # get a bunch of marbles 
    marbles = np.array([sample_marble(box) for ii in range(num_samples)])
    # compute fraction of marbles of desired color 
    return np.sum(marbles == color) / num_samples

In [3]:
probability_of_color("black", box, num_samples=50000)

0.49781999999999998

### Exercise 1 - More Marbles 
*** 

Suppose now we add a third color to the mix.  Bag 1 now contains 6 white marbles, 4 black marbles, and 5 gray marbles.  Bag 2 now contains 3 white marbles, 7 black marbles, and 5 gray marbles.  The probability of grabbing the first bag from the box is still twice the probability of grabbing the second bag.  

What is the probability of drawing a gray marble from the bag?  

**Part A**: First try to do this from intuition, then work the problem by hand. 

***
**Solution**: Since there is a $\frac{1}{3}$ probability of drawing a gray marble from each bag, it seems like the probability should be $\frac{1}{3}$.  Let's check this using the law of total probability.  

Let $G$ be the event that we select a gray marble, and $B_1$ and $B_2$ be the events that we select Bags 1 and 2 from the box, respectively.  We then have 

$$
P(G) = P(G \mid B_1)~P(B_1) + P(G \mid B_2)~P(B_2) = \frac{1}{3}\cdot\frac{2}{3} + \frac{1}{3}\cdot\frac{1}{3} = \frac{2}{9} + \frac{1}{9} = \frac{3}{9} = \frac{1}{3} \quad \checkmark
$$
***

**Part B**: Next, copy and paste the code from the example above and modify it to estimate the probability that you just computed and check your work. 

***
**Solution**: 

In [4]:
bag1 = {'marbles' : np.array(["black", "white", "gray"]), 'probs' : np.array([4/15, 6/15, 5/15])}
bag2 = {'marbles' : np.array(["black", "white", "gray"]), 'probs' : np.array([7/15, 3/15, 5/15])}
box  = {'bags' : np.array([bag1, bag2]), 'probs' : np.array([2/3, 1/3])}

def sample_marble(box):
    # randomly choose a bag 
    bag = np.random.choice(box['bags'], p = box['probs'])
    # randomly choose a marble 
    return np.random.choice(bag['marbles'], p = bag['probs'])

def probability_of_color(color, box, num_samples=1000):
    # get a bunch of marbles 
    marbles = np.array([sample_marble(box) for ii in range(num_samples)])
    # compute fraction of marbles of desired color 
    return np.sum(marbles == color) / num_samples

In [5]:
probability_of_color("gray", box, num_samples=100000)

0.33413999999999999

Looks like a good approximation of $\frac{1}{3}$ to me! 
***

### Exercise 2 - Estimating Conditional Probabilities 
***

Suppose you roll a fair die two times.  Let $A$ be the event "the sum of the throws equals 4" and $B$ be the event "at least one of the throws is a $3$"

**Part A**: Compute (by hand) the probability that the sum of the throws equals 4 _given_ that at least one of the throws is a 3.  That is, compute $P(A \mid B)$. 

***
**Solution**: We want to compute 

$$
P(A \mid B) = \dfrac{P(A \cap B)}{P(B)}
$$

The intersection of the two events is the set $A \cap B = \{(3,1), (1,3)\}$.  Each possible roll combination occurs with probability $\frac{1}{6}\cdot\frac{1}{6} = \frac{1}{36}$. Summing over the possible outcomes gives 

$$
P(A \cap B) = \frac{1}{36} + \frac{1}{36} = \frac{1}{18}
$$

The probability of at least one of the throws being a $3$ can be computed in a number of ways.  Let $E_1$ be the event that we roll a 3 on the first die and $E_2$ be the event that we roll a 3 on the second die. Notice that $E_1$ and $E_2$ are not disjoint because we could roll a 3 on _both_ dice.  So we have 

$$
P(B) = P(E_1 \cup E_2) = P(E_1) + P(E_2) - P(E_1 \cap E_2) = \frac{1}{6} + \frac{1}{6} - \frac{1}{6}\cdot\frac{1}{6} = \frac{11}{36}
$$

Plugging this into the definition of conditional probability gives 

$$
P(A \mid B) = \dfrac{P(A \cap B)}{P(B)} = \frac{1/18}{11/36} = \frac{2}{11}
$$

***

**Part B**: Let's see if we can write a simple simulation to confirm our result.  The following code runs a simulation to estimate $P(A)$, i.e. the probability that if you roll a fair six-sided die twice the result will sum to 4.  Your job is to modify the code so that it estimates the conditional probability $P(A \mid B)$. **Hint**: Think about the definition of conditional probability.

***
**Solution**: 

In [6]:
die = np.array([1,2,3,4,5,6])

num_samples = 100000
roll1 = np.random.choice(die, size=num_samples)
roll2 = np.random.choice(die, size=num_samples)
sum_to_four = (roll1 + roll2) == 4
sum_to_four_prob = np.sum(sum_to_four)/num_samples
print("The probability of rolling a sum-to-four is approximately {:.3f}".format(sum_to_four_prob))

The probability of rolling a sum-to-four is approximately 0.085


In [7]:
die = np.array([1,2,3,4,5,6])

num_samples = 500000
roll1 = np.random.choice(die, size=num_samples)
roll2 = np.random.choice(die, size=num_samples)

sum_to_four = (roll1 + roll2) == 4
at_least_one_3 = np.logical_or(roll1 == 3, roll2 == 3)
four_given_three_prob = np.sum(np.logical_and(sum_to_four, at_least_one_3)) / np.sum(at_least_one_3)
print("The probability of rolling a sum-to-four given at least one 3 is approximately {:.3f}".format(four_given_three_prob))

The probability of rolling a sum-to-four given at least one 3 is approximately 0.181


This looks very close to $\dfrac{2}{11} \approx 0.\overline{18}$!

***

### Exercise 3 - The Ol' Marble Switcharoo
***
A marble is drawn at random from a bag containing one black and one white marble.  If the white marble is drawn it is put back into the bag.  If the black marble is drawn, it is returned to the bag along with two **more** black marbles.  Then a second draw is made. What is the probability a black marble was drawn on **both** the first and the second draws?  

**Part A**: Let's do this one in reverse order.  First, see if you can write a simple simulation to estimate this probability. 

***
**Solution**: 

In [11]:
marbles = np.array(["black", "white"])

num_samples = 100000

# draw the first marble 
m1 = np.random.choice(marbles, size=num_samples)

# function to draw second marble based on result of first marble 
draw_from_second_bag = lambda m: np.random.choice(marbles) if m=="white" else np.random.choice(marbles, p=[3/4, 1/4])

# make it a vectorized function so it runs super-duper fast 
vectorized_second_draw = np.vectorize(draw_from_second_bag)

# draw second marble based on first marble 
m2 = vectorized_second_draw(m1)

# compute ratio of double-black to number of samples 
both_black_prob = np.sum(np.logical_and(m1=="black", m2=="black")) / num_samples
print("The probability that both marbles are black is approximately {:.3f}".format(both_black_prob))

The probability that both marbles are black is approximately 0.374


***

**Part B**: Now carry out the computation by hand and see if they agree. 

***
**Solution**: Let $B_1$ be the event that a black marble is drawn first and $B_2$ be the event that a black marble is drawn second.  We want to compute $P(B_1 \cap B_2)$.  We'll do this using the product rule 

$$
P(B_1 \cap B_2) = P(B_2 \mid B_1)~P(B_1) 
$$

Now, clearly the probability of a black marble being drawn first is $P(B_1) = \frac{1}{2}$.  The conditional probability that a black marble is drawn second _given_ that a black marble was drawn first is $P(B_2 \mid B_1) = \frac{3}{4}$. 

Then the probability of both marbles being black is 

$$
P(B_1 \cap B_2) = P(B_2 \mid B_1) ~ P(B_1) = \frac{3}{4}\cdot\frac{1}{2} = \frac{3}{8}
$$

which seems to agree very well with our simulation!
***

### Exercise 4: Bayes' Rule and The Monte Hall Problem 
***


>Suppose you're on a game show, and you're given the choice of three doors: Behind one door is a car; behind the others, goats. You pick a door, say No. 1, and the host, who knows what's behind the doors, opens another door, say No. 3, which has a goat. He then says to you, "Do you want to pick door No. 2?" Is it to your advantage to switch your choice?

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/3f/Monty_open_door.svg/2000px-Monty_open_door.svg.png",width=801,height=400>

**Part A**: What does your intuition say?  Is it in your best interest to switch? 

***
**Solution**:  It is in your interest to switch.  It turns out that if you don't switch doors you have a 1/3 probability of winning the car.  If you do switch doors you have a 2/3 probability.  We'll prove this using Bayes rule in the next part, but if this seems sketchy to you, consider a modified game where there are 100 doors with 99 goats and one car.  After making your guess the host (knowing where the car is) opens 98 doors that have goats behind them.  Now does your intuition change? 
***

**Part B**: Let's assume that you pick door number 1 and Monty opens door number 3.  The question then is whether you stick with door number 1 or switch to door number 2. Let $D_i$ be the event that the car is actually behind door $i$.  Let $H$ be the event that Monty opens door number 3. First, compute $P(H \mid D_i)$ for $i=1,2,3$.  You'll need to think carefully about the particular situation described above.

***
**Solution**: The likelihoods, given our assumptions, are as follows: 

- $D_1$: If the car is behind Door 1 then it doesn't matter which of Doors 2 and 3 that Monte opens.  Thus this probability is $P(H \mid D_1) = \frac{1}{2}$

- $D_2$: If the car is behind Door 2 then Monte must open Door 3.  Thus this probability is $P(H \mid D_2) = 1$

- $D_3$: If the car is behind Door 3 then there is no way that Monte will open Door 3.  Thus this probability is $P(H \mid D_3) = 0$

***

**Part C**: Use your results from **Part B** and the Law of Total Probability to compute $P(H)$

***
**Solution**: Assuming that it is equally likely that the car is behind any door, we have $P(D_i) = \frac{1}{3}$ for $i=1, 2, 3$. We then have 

\begin{eqnarray}
\nonumber P(H) &=& P(H \mid D_1)P(D_1) + P(H \mid D_3)P(D_3) + P(H \mid D_3)P(D_3) \\
\nonumber      &=& \frac{1}{2}\cdot\frac{1}{3} + 1\cdot\frac{1}{3} + 0\cdot\frac{1}{3} \\ 
\nonumber      &=& \frac{1}{2}
\end{eqnarray}

***

**Part D**: Now, use Bayes' Rule to compute $P(D_i \mid H)$ for $i=1$ and $2$ (because these are the doors we care about). 

***
**Solution**: From Bayes' Rule, we have 

$$
p(D_1 ~|~ H) = \frac{P(H \mid D_1)P(D_1)}{P(H)} = \frac{\frac{1}{2} \cdot \frac{1}{3}}{\frac{1}{2}} = \frac{1}{3} ~~~~~~~~ \textrm{and} ~~~~~~~~
p(D_2 ~|~ H) = \frac{P(H \mid D_2)P(D_2)}{P(H)}  = \frac{1 \cdot \frac{1}{3}}{\frac{1}{2}} = \frac{2}{3}
$$

Thus if we switch we have a probability of $\frac{2}{3}$ of winning the car. 

***

**Part E**: Write some simple code to simulate the Monte Hall problem and verify your results from **Parts A-D**. 

In [29]:
def make_a_deal(switch=True):
    doors = list(range(3))
    car = np.random.choice(doors)
    first_choice = np.random.choice(doors)
    montes_options = list(set(doors)-set([car])-set([first_choice]))
    goat = np.random.choice(montes_options)
    final_choice = (set(doors)-set([goat])-set([first_choice])).pop() if switch else first_choice
    return final_choice == car

def monte_hall_sim(switch=True, num_trials=int(1e3)): 
    winners = np.sum([make_a_deal(switch) for kk in range(num_trials)])
    state = "switching" if switch else "not switching"
    print("P(winning by "+state+" = {:.4f}".format(winners/num_trials))

In [30]:
monte_hall_sim(switch=True, num_trials=int(1e5))

P(winning by switching = 0.6683


In [31]:
monte_hall_sim(switch=False, num_trials=int(1e5))

P(winning by not switching = 0.3313
